In [1]:
import requests
import json
import codecs
import sys, time
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
username = "7569524a-048e-456e-aae6-34f035c69503"
password = "I2xIGAJJ2RjK"
headers = {'Content-Type' : "application/json"}

In [15]:
print "\nCreating custom mmodel..."
data = {"name" : "Custom model #2", "base_model_name" : "ja-JP_BroadbandModel", "description" : "300312-01 custom model and Add a single user word"}
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations"
jsonObject = json.dumps(data).encode('utf-8')
resp = requests.post(uri, auth=(username,password), verify=False, headers=headers, data=jsonObject)

print "Model creation returns: ", resp.status_code
if resp.status_code != 201:
   print "Failed to create model"
   print resp.text


Creating custom mmodel...
Model creation returns:  201


In [25]:
respJson = resp.json()
customID = respJson['customization_id']
print "Model customization_id: ", customID

Model customization_id:  f7b13892-8070-4af3-89f8-fcddb1c33f3c


In [26]:
corpus_file = "../corpus_utf8.txt"
corpus_name = "corpus1"

In [27]:
print "\nAdding corpus file..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/corpora/"+corpus_name
with open(corpus_file, 'rb') as f:
   r = requests.post(uri, auth=(username,password), verify=False, headers=headers, data=f)

print "Adding corpus file returns: ", r.status_code
if r.status_code != 201:
   print "Failed to add corpus file"
   print r.text


Adding corpus file...
Adding corpus file returns:  400
Failed to add corpus file
{
   "code": 400,
   "code_description": "Bad Request",
   "error": "Corpus 'corpus1' already exists - change its name, remove existing corpus before adding new one, or overwrite existing corpus by setting 'allow_overwrite' flag to true"
}


In [28]:
print "Checking status of corpus analysis..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/corpora/"+corpus_name
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
respJson = r.json()
status = respJson['status']
time_to_run = 10
while (status != 'analyzed'):
    time.sleep(10)
    r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
    respJson = r.json()
    status = respJson['status']
    print "status: ", status, "(", time_to_run, ")"
    time_to_run += 10

print "Corpus analysis done!"

Checking status of corpus analysis...
Corpus analysis done!


In [11]:
print "\nListing words..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/words?sort=count"
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
print "Listing words returns: ", r.status_code
file=codecs.open(customID+".OOVs.corpus", 'wb', 'utf-8')
file.write(r.text)
print "Words list from added corpus saved in file: "+customID+".OOVs.corpus"


Listing words...
Listing words returns:  200
Words list from added corpus saved in file: 645137fe-e93c-462f-8ea0-699b6a6e6898.OOVs.corpus


In [43]:
print "\nAdding multiple words..."
data = {"words" : [{"word" : "ごばん", "sounds_like" : ["ゴバン"], "display_as" : "5番"},{"word" : "にじゅよんごう", "sounds_like" : ["ニジュヨンゴウ"], "display_as" : "二十四号"}, {"word" : "ふくむらのりちか", "sounds_like" : ["フクムラノリチカ"], "display_as" : "福村教親"}]}
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/words"
jsonObject = json.dumps(data).encode('utf-8')
r = requests.post(uri, auth=(username,password), verify=False, headers=headers, data=jsonObject)

print "Adding multiple words returns: ", r.status_code


Adding multiple words...
Adding multiple words returns:  201


In [44]:
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
respJson = r.json()
status = respJson['status']
print "Checking status of model for multiple words..."
time_to_run = 10
while (status != 'ready'):
    time.sleep(10)
    r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
    respJson = r.json()
    status = respJson['status']
    print "status: ", status, "(", time_to_run, ")"
    time_to_run += 10

print "Multiple words added!"

Checking status of model for multiple words...
status:  ready ( 10 )
Multiple words added!


In [46]:
print "\nListing words..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/words?word_type=user&sort=alphabetical"
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)

print "Listing user-added words returns: ", r.status_code
file=codecs.open(customID+".OOVs.user", 'wb', 'utf-8')
file.write(r.text)
print "Words list from user-added words saved in file: "+customID+".OOVs.user"


Listing words...
Listing user-added words returns:  200
Words list from user-added words saved in file: f7b13892-8070-4af3-89f8-fcddb1c33f3c.OOVs.user


In [47]:
print "\nTraining custom model..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID+"/train"
data = {}
jsonObject = json.dumps(data).encode('utf-8')
r = requests.post(uri, auth=(username,password), verify=False, data=jsonObject)

print "Training request returns: ", r.status_code
if r.status_code != 200:
   print "Training failed to start - exiting!"


Training custom model...
Training request returns:  200


In [48]:
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations/"+customID
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
respJson = r.json()
status = respJson['status']
time_to_run = 10
while (status != 'available'):
    time.sleep(10)
    r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
    respJson = r.json()
    status = respJson['status']
    print "status: ", status, "(", time_to_run, ")"
    time_to_run += 10

print "Training complete!"

print "\nGetting custom models..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations"
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)

print "Get models returns: ", r.status_code
print r.text

status:  training ( 10 )
status:  available ( 20 )
Training complete!

Getting custom models...
Get models returns:  200
{"customizations": [
   {
      "owner": "1f3ca766-8b0f-4a3c-a024-6f2c2e20e18e",
      "base_model_name": "ja-JP_BroadbandModel",
      "customization_id": "645137fe-e93c-462f-8ea0-699b6a6e6898",
      "dialect": "ja-JP",
      "versions": ["ja-JP_BroadbandModel.v09-05202016.06202016"],
      "created": "2018-04-10T05:40:09.422Z",
      "name": "Custom model #1",
      "description": "300312-01 custom model",
      "progress": 100,
      "language": "ja-JP",
      "status": "available"
   },
   {
      "owner": "1f3ca766-8b0f-4a3c-a024-6f2c2e20e18e",
      "base_model_name": "ja-JP_BroadbandModel",
      "customization_id": "f7b13892-8070-4af3-89f8-fcddb1c33f3c",
      "dialect": "ja-JP",
      "versions": ["ja-JP_BroadbandModel.v09-05202016.06202016"],
      "created": "2018-04-10T06:59:08.784Z",
      "name": "Custom model #2",
      "description": "300312-01 custo

In [49]:
print "\nGetting custom models..."
uri = "https://stream.watsonplatform.net/speech-to-text/api/v1/customizations"
r = requests.get(uri, auth=(username,password), verify=False, headers=headers)
print "Get models returns: ", r.status_code
print r.text


Getting custom models...
Get models returns:  200
{"customizations": [
   {
      "owner": "1f3ca766-8b0f-4a3c-a024-6f2c2e20e18e",
      "base_model_name": "ja-JP_BroadbandModel",
      "customization_id": "645137fe-e93c-462f-8ea0-699b6a6e6898",
      "dialect": "ja-JP",
      "versions": ["ja-JP_BroadbandModel.v09-05202016.06202016"],
      "created": "2018-04-10T05:40:09.422Z",
      "name": "Custom model #1",
      "description": "300312-01 custom model",
      "progress": 100,
      "language": "ja-JP",
      "status": "available"
   },
   {
      "owner": "1f3ca766-8b0f-4a3c-a024-6f2c2e20e18e",
      "base_model_name": "ja-JP_BroadbandModel",
      "customization_id": "f7b13892-8070-4af3-89f8-fcddb1c33f3c",
      "dialect": "ja-JP",
      "versions": ["ja-JP_BroadbandModel.v09-05202016.06202016"],
      "created": "2018-04-10T06:59:08.784Z",
      "name": "Custom model #2",
      "description": "300312-01 custom model and Add a single user word",
      "progress": 100,
      "lan